# 1.讀取和寫入文字格式資料

### pandas提供一對用來將表格式資料讀入變成DataFrame物件的函式。 read_csv 和 read_table 應該會是最常用的兩個！

![jupyter](./padnas的解析函式.PNG)

![jupyter](解析函式可選參數.PNG)

***

### 一個逗號分隔的小文字檔案範例:

In [7]:
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


#### 由於是用逗號分隔，所以利用read_csv將它讀入一個DataFrame中:

In [8]:
import pandas as pd

In [9]:
df=pd.read_csv('ex1.csv')

In [10]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 也可改用read_table指定分隔符號:

In [12]:
pd.read_csv('ex1.csv',sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


***

#### 檔案裡不一定會有標題(header)列:

In [13]:
!type ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


#### 若想將檔案讀入，有幾種作法。可讓pandas自行指定預設的欄名稱，或是自行指定:

In [14]:
pd.read_csv('ex2.csv',header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
pd.read_csv('ex2.csv',names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


***

#### 若是想讓message欄變成回傳的DataFrame的index的話，可用index_col參數，指定想要index為4的那一欄，或是指定名稱為'message'的那一欄變成index:

In [17]:
names=['a','b','c','d','message']

In [19]:
pd.read_csv('ex2.csv',names=names,index_col=4)

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [20]:
pd.read_csv('ex2.csv',names=names,index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


***

#### 假設想一多個欄位的值，做出階層式index，就將要用的欄編號或名稱以list傳入:

In [21]:
!type csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [23]:
parsed=pd.read_csv('csv_mindex.csv',index_col=['key1','key2'])

In [25]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

***

#### 某些情況下，一個資料表可能沒有固定的分隔符號，只使用空白或其他的東西做分隔:

In [26]:
list(open('ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

#### 上面資料的分隔式由數個空白組成，可傳一個正規表達式給read_table，當作是分隔符號的識別。此處的正規表達式可寫成\s+:

In [28]:
result=pd.read_table('ex3.txt',sep='\s+')

In [29]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


#### 因為欄位名稱那列的數量比資料列少一個，read_table推斷第一欄應該是DataFrame的索引。

***

### 用skiprows跳過檔案中的某些列:

In [31]:
!type ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [32]:
pd.read_csv('ex4.csv',skiprows=[0,2,3]) #跳過第一、三和四列

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


***

### 遺失值通常可能是不存在(空字串)，也可能是一些標記值。pandas預設會用常用的標記值，例如NA或NULL來當作遺失標記值:

In [33]:
!type ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [34]:
result=pd.read_csv('ex5.csv')

In [35]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [36]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


### na_values參數設定為一組字串，資料值符合字串者當作遺失值:

In [37]:
result=pd.read_csv('ex5.csv',na_values=['NULL'])

In [38]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


### 用dict格式可為每個欄指定不同的NA標記值:

In [39]:
sentinels={'message':['foo','NA'],'something':['two']}

In [40]:
result=pd.read_csv('ex5.csv',na_values=sentinels)

In [41]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


***

![jupyter](部分read_csv和read_table函式參數.PNG)

![jupyter](部分read_csv和read_table函式參數1.PNG)

![jupyter](部分read_csv和read_table函式參數2.PNG)

***

## 分段讀取文字檔

#### 若是遇到要讀很大的檔案，或是試圖想用對的參數去讀很大的檔案時，可先讀一小段內容，或每次讀一小段檔案內容進到程式中處裡。

### 把pandas顯示範圍設定的更精簡一點:

In [42]:
pd.options.display.max_rows=10 

In [43]:
result=pd.read_csv('ex6.csv')

In [44]:
result  #只印出前五後五總共十行。

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


### 若只想讀幾列的話(不想讀整個檔案)，可直接指定rows:

In [45]:
pd.read_csv('ex6.csv',nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### 若想分段讀檔的話，就用chunksize，指定給它的值是一次想讀幾行:

In [46]:
chunker=pd.read_csv('ex6.csv',chunksize=1000)

In [47]:
chunker

#### read_csv會回傳一個TextParser物件，讓你可根據設定的chunksize，用疊代的方法每次處裡檔案的一部分。

***

## 寫出文字資料

#### 資料也可以匯出成分隔符號的格式。

In [48]:
data=pd.read_csv('ex5.csv')

In [49]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


### 使用DataFrame的to_csv方法，可以把資料寫成一個逗號分隔的檔案:

In [50]:
data.to_csv('out.csv')

In [51]:
!type out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


### 改用其他分隔符號(寫到sys.stdout，這樣就會把文字改為顯示在終端機上):

In [52]:
import sys

In [53]:
data.to_csv(sys.stdout,sep='|') #此時遺失值在輸出裡會以空字串的樣子呈現

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


### 遺失值在輸出裡以空字串的樣子呈現時，可用其他的標記值取代空字串:

In [54]:
data.to_csv(sys.stdout,na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


### 若沒有特別指定，預設上列標籤和欄標籤都會被寫出，特別指定的話，兩者都可以不被寫出:

In [56]:
data.to_csv(sys.stdout,index=False,header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


### 可以選擇只寫出一部分欄，順序可指定:

In [57]:
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### Series也有to_csv方法:

In [58]:
dates=pd.date_range('1/1/2000',periods=7)

In [61]:
import numpy as np

In [62]:
ts=pd.Series(np.arange(7),index=dates)

In [63]:
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [64]:
ts.to_csv('tseries.csv')

In [65]:
!type tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


***

## 分隔符號的使用

#### 使用像pandas.read_table這樣的函式，面對磁碟上的不同表格式資料，大部分都能載入。某些情況下得額外手動處裡，例如一到多行有錯的檔案。

###  基本的工具處裡檔案:

In [67]:
!type ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


### 若是以單一字元作為分隔符號，可使用Python內建的csv方法。使用方法是將已打開的檔案或類似檔案的物件傳給csv.reader:

In [68]:
import csv

In [69]:
f=open('ex7.csv'reader)

In [70]:
reader=csv.reader(f)

### 面對檔案一樣地對reader進行疊代的話，會看到值被移除引號並組成tuple:

In [71]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


### 將檔案的列讀成一個個list

In [72]:
with open('ex7.csv') as f:
    lines=list(csv.reader(f))

### 將列區分為標題列和資料列:

In [76]:
header,values=lines[0],lines[1:]

### 用字典綜合運算式和zip(*values)，把欄資料建成字典型式，此動作可把列轉為欄:

In [77]:
data_dict={h:v for h,v in  zip(header,zip(*values))}

In [78]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

#### CSV檔案內容格式形形色色，若想定義新的分隔符號格式、定義字串引號或是行尾結束標記，可自訂一個簡單的csv.Dialect類別:

In [ ]:
class my_dialect(csv.Dialect): 
    lineterminator = '\n' 
    delimiter = ';' 
    quotechar = '"' 
    quoting = csv.QUOTE_MINIMAL

In [ ]:
reader=csv.reader(f,dialect=my_dialect)

#### 也可透過參數指定csv.reader的CSV dialect參數:

In [ ]:
reader = csv.reader(f, delimiter='|')

#### 若要手動寫出資料到分隔符號檔案，可呼叫csv.writer。它可接受一個已開啟、可寫的檔案物件，與csv.reader一樣有dialect和格式選項:

In [ ]:
with open('mydata.csv', 'w') as f: 
    writer = csv.writer(f, dialect=my_dialect) 
    writer.writerow(('one', 'two', 'three')) 
    writer.writerow(('1', '2', '3')) 
    writer.writerow(('4', '5', '6')) 
    writer.writerow(('7', '8', '9'))

#### 若要對付更複雜或是多字元分隔符號檔案，csv模組也無法應付。此情況不得不拆解每一行，使用字串split方法或正規表達式re.split方法來進行清理。

![jupyter](csv_dialect選項.PNG)

***

## JSON資料

#### JavaScript Object Notation 變成網路瀏覽器及其他應用程式，透過HTTP傳送資料的標準格式之一，比像CSV這種表格式文字格式更有彈性。

### JSON範例:

In [6]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [11]:
obj

'\n{"name": "Wes",\n "places_lived": ["United States", "Spain", "Germany"],\n "pet": null,\n "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},\n              {"name": "Katie", "age": 38,\n               "pets": ["Sixes", "Stache", "Cisco"]}]\n}\n'

#### 除了null和其他一些細微差別(例如在list的結尾不能放分逗號)之外，JSON的程式碼和Python程式碼其實長得很像。基本的型態有物件(dict)、陣列(list)、字串、數字、布林和null。一個物件中所有的key都必須式字串。Python中有數個用來讀寫JSON的函式庫，以下選用json函式庫。

In [1]:
import pandas as pd

In [2]:
import json

### 把一個JSON字串轉成Python格式，可呼叫 json.loads:

In [7]:
result=json.loads(obj)

In [8]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

### json.dumps則是用來將Python的物件轉回JSON:

In [9]:
asjson=json.dumps(result)

In [10]:
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

#### 要把一個或一堆的JSON物件轉成DataFrame或其他資料結構，得看分析的需求決定。
### 可將dict的list(從JSON件轉來的)傳給DataFrame的建構子，同時指定選用那些資料:

In [13]:
siblings=pd.DataFrame(result['siblings'],columns=['name','age'])

In [14]:
siblings

,name,age
0,Scott,30
1,Katie,38


### pandas.read_json可自動轉換JSON資料集到一個Series或DataFrame中:

In [15]:
!type example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


### pandas.read_json的預設選項，是將在JSON陣列中的每一個物件轉為表格中的一列:

In [17]:
data=pd.read_json('example.json')

In [18]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


### 若需要從pandas匯出資料到JSON的話，可選擇使ˇ用Series和DataFrame都有的 to_json 方法:

In [19]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [20]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


***

## XML和HTML:從網站抓資料

#### Python有許多函式庫，可用來從被普遍使用的HTML和XML格是抓取和寫出資料。函式庫有lxml、Beautiful Soup和html5lib。雖然lxml相對來說比較快，但其他的函式更有辦法處裡有問題的HTML或XM檔案。

#### pandas有內建的read_html函式，利用像lxml和Beautiful soup等函式庫來自動解析HTML檔案中的資料，轉為DataFrame物件。下方以美國聯邦存款公司的政府部分下載一個HTML檔做範例，裡面有銀行出錯的紀錄。

### pandas.read_html函式有一堆選項，但預設會搜尋並企圖解析被<table>標籤包含的表格式資料。出來的結果將是DataFrame物件的list:

In [24]:
tables=pd.read_html("fdic_failed_bank_list.html")

In [25]:
tables

[                                  Bank Name             City  ST   CERT  \
 0                               Allied Bank         Mulberry  AR     91   
 1              The Woodbury Banking Company         Woodbury  GA  11297   
 2                    First CornerStone Bank  King of Prussia  PA  35312   
 3                        Trust Company Bank          Memphis  TN   9956   
 4                North Milwaukee State Bank        Milwaukee  WI  20364   
 5                    Hometown National Bank         Longview  WA  35156   
 6                       The Bank of Georgia   Peachtree City  GA  35259   
 7                              Premier Bank           Denver  CO  34112   
 8                            Edgebrook Bank          Chicago  IL  57772   
 9                    Doral Bank  En Espanol         San Juan  PR  32102   
 10        Capitol City Bank & Trust Company          Atlanta  GA  33938   
 11                  Highland Community Bank          Chicago  IL  20290   
 12         

In [29]:
len(tables)

1

In [31]:
failures=tables[0]

In [32]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


### 此處開始可做資料清理和分析，如計算每年的銀行出錯次數:

In [33]:
close_timestamps=pd.to_datetime(failures['Closing Date'])

In [35]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

***

## 用lxml.objectify解析XML

#### XML(eXtensible Markup Language)是另外一種常用來支援階層式、巢式以及代metadata的資料結構。XML和HTML兩者結構相似，但XML的使用更為普遍。

### 下列從紐約大都會運輸管理局公開的一系列關於巴士和火車服務的資料集作為範例:

#### 資料效能資料是放在一組XML檔案裡，資料火車和巴士都記錄在不同的檔案之中，該檔案用一些XML紀錄表是每月的資料。

### 使用lxml.objectify的話，可以解析檔案，並使用getroot取得該檔XML檔案中的根結點參考:

In [36]:
from lxml import objectify

path = 'mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

### root.INDICATOR回傳的是一個產生器，此產生器會產生出所 < INDICATOR > XML元素，對每筆紀錄，我們ㄉ可以產出標籤名稱與填充資料值得一個dict(除了少部分標記之外):

In [37]:
data=[]

In [39]:
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

### 最後，將該dict組成的list轉為DataFrame:

In [40]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


### 可能碰到的XML資料比範例的複雜許多，每個標籤還可以自帶metadata。就像HTML連結標籤，也是個合法的XML:

In [41]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

### 現在可任意存取標籤中的欄位(像href)或是連結的文字:

In [42]:
root

<Element a at 0x25a74018a48>

In [44]:
root.get('href')

'http://www.google.com'

In [46]:
root.text

'Google'

***

# 2.二進位資料格式

#### 使用Python內建pickle序列化工具，是一種儲存二進位格式資料(也稱為 序列化-serialization)最簡單的方式之一。

### pandas物件都有to_pickle方法，用來將資料以pickle格式寫入磁碟:

In [2]:
import pandas as pd

In [3]:
frame=pd.read_csv('ex1.csv')

In [4]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
frame.to_pickle('frame_pickle') #frame_pickle檔案被新增

### 可使用內建的pickle讀取檔案中任何被"pickle"過的物件，或是使用pandas.read_pickle:

In [6]:
pd.read_pickle('frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 建議只有在需要短暫儲存資料的情況才使用pickle，由於很難保證隨著時間推移，pickle的格式不會跟著改；今天pickle過的資料，明天新版的函式庫不一定適用。



#### pandas內建支援兩種二進位檔案格式:HDF5和MessagePcak。
#### 其他能配合pandas和NumPy使用的儲存格是包括:bcolz和Feather。

***

## HDF5格式

#### HDF5是種良好的檔案格式，用來儲存大量的科學陣列資料。"HDF"指的是階層式資料格式(hierarchical data format)。每個HDF5檔案可儲存多個資料集，並支援metadata。和其他簡單的格式相比，HDF5利用多種壓縮模式支援即時壓縮，使得有重複特徵的資料，可用更有效率的方法儲存。 

#### 由於可有效率地存取大型陣列中的小區塊，所以在處裡體積大到放不進記憶體的資料集時，HDF5是個很好的選擇。PyTables或h5py函式庫皆能直接存取HDF5檔案。

### pandas提供了能簡化儲存Series和DataFrame物件的高階介面HDFStore類別，用起來像dict，可把底層細節都處裡調:

In [9]:
import numpy as np

In [10]:
frame=pd.DataFrame({'a':np.random.randn(100)})

In [11]:
store=pd.HDFStore('mydata.h5')

In [12]:
store['obj1']=frame

In [13]:
store['obj1_col']=frame['a']

In [14]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

### 在HDF5檔案中的物件，可用類似dict的API存取:

In [15]:
store['obj1']

,a
0,1.167224
1,-0.148733
2,0.916436
3,0.546529
4,0.205134
5,0.123481
6,-1.370863
7,-0.667895
8,0.196001
9,0.794762


### HDF5支援'fixed'和'table'兩種儲存方法，後者速度通常慢一點，但支援特殊的查詢語法:

#### put是store['obj2']=frame的顯式寫法，可指定儲存格式等參數。

In [16]:
store.put('obj2',frame,format='table') 

In [17]:
store.select('obj2',where=['index >= 10 and index <= 15 '])

,a
10,2.561940
11,0.375329
12,-1.480334
13,-0.033434
14,-1.602084
15,-1.050946


HDF5並不是一種資料庫，它適合在寫一次但會讀取很多次的資料集合。雖然任何時間都可以將資料寫到檔案，但是多個同時的寫出動作可能損毀檔案。

***

## 讀取Microsoft Excel檔案

#### pandas也支援從Excel2003(和之後版本)檔案中讀取資料表格，可使用ExcelFile類別或式pandas.read_excel函式。

### ExcelFile接收xls或xlsx檔案路徑，然後建立實例:

In [26]:
xlsx=pd.ExcelFile('ex1.xlsx')

### 儲存在工作表中的資料被解析完後，會轉為DataFrame:

In [27]:
pd.read_excel(xlsx,"Sheet1")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 如果想從一個檔案中讀取多個工作列表，建立一個ExcelFile會比較快，但也可簡單地重複傳遞檔案給pandas.read_excel:

In [31]:
frame=pd.read_excel('ex1.xlsx','Sheet1')

In [32]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 若想把pandas資料以Excel格式寫出，首先建立ExcelWriter，然後使用pandas物件的to_excel方法，把資料寫給ExcelWriter:

In [33]:
writer=pd.ExcelWriter('ex2.xlsx')

In [34]:
frame.to_excel(writer,'Sheet1')

In [35]:
writer.save()

### 若不想使用ExcelWriter，也可把路徑傳給to_excel:

In [36]:
frame.to_excel('ex2.xlsx')

***

# 3.使用Web API

許多網站透過JSON或其他格式，提供公開的API資料點。Python中有數種方法可以存取這些API；其中最容易的方法是使用requests套件。

### 以從github上查詢pandas最新的30個issue，我們可利用額外的requests函式庫，做出一個GET HTTP requests進行查詢:

In [37]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

### 上面代表回應結果的物件中，有個JSON方法，會將JSON轉為Python元生物件後，用字典型態回傳:

In [38]:
data=resp.json()

In [39]:
data[0]['title']

'Weird performance characteristics of resampled quantile() function (100 times slower)'

在data中的每個元素，都是一個字典型態，其中包含了所有在GITHUB issue 頁面上的所有資料(除了註解)。

### 現在可把data直接傳給DataFrame，並指定要用的欄位:

In [40]:
issues=pd.DataFrame(data,columns=['number','title','labels','state'])

In [41]:
issues

,number,title,labels,state
0,26150,Weird performance characteristics of resampled...,[],open
1,26149,Option to give service account in case of Goog...,[],open
2,26148,Fix Type Annotation in pandas/core/api.py,[],open
3,26147,Fix Type Annotation in pandas.core.accessor,[],open
4,26146,Fix Type Annotation in pandas/core/accessor.py,[],open
5,26145,to_excel column name can't output to excel,[],open
6,26144,to_hdf5 with pd.Int32Dtype gives ValueError: c...,[],open
7,26141,Added example to docstring,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
8,26140,BUG: update cython to fix segfault accessing _...,[],open
9,26139,Doc for HDFStore compression unclear on what t...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open


***

## 4.使用資料庫

在商業環境中，多數的資料並不會儲存在純文字或Excel檔案中。像SQL關聯式資料庫就被廣泛運用。選擇資料庫的條件，通常是它的效能、資料完整性以及應用所需的彈性。

#### 把資料從SQL中載入到DataFrame是很簡單的一件事，pandas有一些函式可簡化這個流程。

### 用Python內建的sqlite3驅動程式，建一個SQLLite資料庫:

In [42]:
import sqlite3

In [43]:
#資料庫
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

### 然後插入幾列資料:

In [44]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

多數的Python SQL驅動程式，在從表中選擇資料時，會回傳tuple組成的一個list。

In [45]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

### 可將這個tuple組成的list傳給DataFrame的建構子，但還需要有欄位名稱，欄位名稱被放在cursor的description屬性中:

In [47]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [48]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


由於上面有點花功夫，可能不想每次進行query都要重做這些動作，Python有一個SQLAlchemy專案，它式一個有名的Python SQL工具集，該工具集對常用的不同的SQL資料庫做了抽象化的工作。

#### pandas有一個叫read_sql的函式，這函式可從一般的SQLAlchemy連線中輕易的讀取資料。

### 用SQLAlchemy讀取同一個SQLite資料庫，並從之前建立的資料表中讀出資料:

In [49]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


***